In [1]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['pandas', 'psycopg2-binary', 'spacy']
pipmain(['install'] + packages)

In [19]:
%%capture
from nltk.tokenize import casual_tokenize
import nltk
nltk.download('averaged_perceptron_tagger')

import pandas as pd
import psycopg2
import spacy
from spacy.matcher import PhraseMatcher
import re

In [3]:
%%capture
!python -m spacy download en_core_web_lg

In [4]:
import spacy
nlp = spacy.load('en_core_web_lg')

Note you need a file with the aws credentials to access the database.   
Here, I am using credentials stored in a file called `aws.py` with the following format:
```
host = 'endpoint_url_string'
port = port_number
user = 'user_string'
password = 'password_string'
database = 'db_name_string'
```

[Amazon RDS — Step-By-Step Guide](https://towardsdatascience.com/amazon-rds-step-by-step-guide-14f9f3087d28)

In [5]:
from aws import host as ahost, port as aport, user as auser, password as apassword, database as adatabase

connection = psycopg2.connect(host = ahost, 
                              port = aport, 
                              user = auser, 
                              password = apassword, 
                              dbname = adatabase)
cursor = connection.cursor()

In [6]:
def read_try(sql):
    try:
        df = pd.read_sql(sql, con=connection)
        return pd.DataFrame() if df.empty else df
    except Exception as e:
        #print("READ ERROR", e)
        return pd.DataFrame()
        
# get all the users from the users table
def read_document_tags_table_from_db():
    sql = 'SELECT * FROM document_tags'
    return read_try(sql)

In [7]:
document_tags = read_document_tags_table_from_db()

In [8]:
document_tags

,allegation_id,title,document_text,url,text_bad,incident_date,allegation_category,allegation_name,organization,nudity_penetration,sexual_harassment_remarks,sexual_humiliation_extortion_or_sex_work,tasers,trespass,racial_slurs,planting_drugs_guns,neglect_of_duty,refuse_medical_assistance,irrational_aggressive_unstable,searching_arresting_minors
0,315627,CRID 315627 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-09-20,Operation/Personnel Violations,Misuse Of Department Equipment / Supplies,CPB,None,None,None,None,None,None,None,None,None,None,None
1,315524,CRID 315524 CR,SUMMARY REPORT DIGEST- COMPLAINT REGISTER INVE...,https://assets.documentcloud.org/documents/440...,True,2006-08-24,Operation/Personnel Violations,Neglect Of Duty,CR,None,None,None,None,None,None,None,None,None,None,None
2,315452,CRID 315452 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-08-31,Operation/Personnel Violations,Neglect Of Duty,CPB,None,None,None,None,None,None,None,None,None,None,None
3,315343,CRID 315343 CR,COMMAND CHANNEL REVIEW A Sasso Emp 14982 NW 1 ...,https://assets.documentcloud.org/documents/457...,None,2006-07-21,False Arrest,Illegal Arrest / False Arrest,CR,False,False,False,False,False,False,False,False,False,False,False
4,315047,CRID 315047 CPB,None,https://assets.documentcloud.org/documents/577...,None,2006-06-11,Traffic,Improper Processing / Reporting / Procedures,CPB,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,1000436,CRID 1000436 CR,REPORT DIGEST LOG NO. TYPE DATE OF REPORT CHI...,https://assets.documentcloud.org/documents/476...,None,2006-04-21,Operation/Personnel Violations,Eeo Investigations,CR,None,None,None,None,None,None,None,None,None,None,None
1679,1000417,CRID 1000417 CR,COMMAND CHANNEL REVIEW COMPLAINT REGISTER INVE...,https://assets.documentcloud.org/documents/457...,None,2006-10-06,False Arrest,Illegal Arrest / False Arrest,CR,None,None,None,None,None,None,None,None,None,None,None
1680,1000301,CRID 1000301 CR,7/18/2018 View Incident Print Logout Help ...,https://assets.documentcloud.org/documents/476...,None,2006-10-07,Operation/Personnel Violations,Neglect Of Duty,CR,None,None,None,None,None,None,None,None,None,None,None
1681,1000227,CRID 1000227 CR (Discovery),1000227 COMMAND CHANNEL REVIEW-SUSTAINED CASE ...,https://assets.documentcloud.org/documents/522...,None,2006-05-22,Operation/Personnel Violations,Misuse Of Department Equipment / Supplies,CR,None,None,None,None,None,None,None,None,None,None,None


Document tags we are looking for:   
(List from Piazza)
- Nudity, Penetration
- sexual harassment, Sexual remarks
- Sexual humiliation, Sexual extortion, Prostitution/sex work
- Tasers, Baton, aggressive physical touch, gun
- Trespass/robbery
- Racial slurs, xenophobic remarks
- Undocumented status- asking for someone’s status, Calling ICE
- Planting drug/guns
- Neglect of duty, failure to serve
- Refusing to provide medical assistance
- “Irrational”, “Aggressive”, “Unstable”
- searching/patting down/arresting minors

In [49]:
test_index = 3
text1 = document_tags['document_text'][test_index]
print(text1)

pdf1 = document_tags['url'][test_index]

COMMAND CHANNEL REVIEW A Sasso Emp 14982 NW 1 32007 DATE INITIATED COMPLAINT REGISTER INVESTIGATION No.2 CHICAGO POLICE DEPARTMENT 315343 12 MARCH 200? INVESTIGATOR SECTION T0: UNIT: Commanding Of?cer Unit IZI-GIS PLEASE SUBMIT THIS COMMAND CHANNEL REVIEW FORM THROUGH CHANNELS WITH THE CASE FILE TO: INTERNAL AFFAIRS DIVISION OFFICE OF PROFESSIONAL STANDARDS DATE RECEIVED BY UNIT I CONCUR [a RECOMMEND OPTIONS BE GRANTED NOT RECOMMEND OPTIONS NOT BE GRANTED REVIEWED BY TITLE A  UNIT - DATE FORWARDED BY UNIT 0 ?Wrdy a 7 DATE RECEIVED BY UNIT  I CONCUR I DO NOT  14/4457: 70%? RECOMMEND OPTIONS BE GRANTED CI RECOMMEND OPTIONS NOT BE GRANTED REVIEWED BYZ UNIT 27/ Maj/6;? 4/ Dem/{Z/ 54?  DATE FORWARDED BY UNIT DATE RECEIWY UNIT [3 I CONCUR RECOMMEND OPTIONS NOT BE GRANTED I DO NOT RECOMMEND OPTIONS SE GRANTED REVIEWED BY I TITLE UNIT I I DATE FORWARDED BY UNIT DATE RECEIVED BY UNIT I CONCUR RECOMMEND OPTIONS NOT BE GRANTED I DO NOT RECOMMEND OPTIONS BE GRANTED REVIEWED BY TITLE UNIT DATE FORW

In [11]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [50]:
print(pdf1)
PDF(pdf1,size=(1000,450))

https://assets.documentcloud.org/documents/4575428/CAR-CR315343-Redacted.pdf


As we can see from the text and the pdf (retrieved from the url), the text version of the pdf is terribly unclean.

- [spaCy text classifier](https://spacy.io/usage/examples#textcat)

In [26]:
# tokenize
def get_tokens(text):
    doc = nlp(text)
    return [token.orth_ for token in doc if not token.is_punct | token.is_space] 

# split by puncuation
def split_by_punc(text):
    return [x.strip() for x in re.split('[,/]', text)]

In [35]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')# the list containing the pharses to be matched
terminology_list = ["Nudity, Penetration", 
                   "sexual harassment, Sexual remarks", 
                   "Sexual humiliation, Sexual extortion, Prostitution/sex work",
                   "Taser, Baton, physical touch, gun",
                   "Trespass/robbery, Racial slurs, xenophobic remarks",
                   "Undocumented status, ICE",
                   "Planting drug, planting guns",
                   "Neglect of duty, failure to serve",
                   "Refusing to provide medical assistance",
                   "Irrational, Aggressive, Unstable",
                   "searching minors/patting down minors/arresting minors"]
# split by punctuation and spaces

In [36]:
for t in terminology_list:
    print(t, ": ", split_by_punc(t))

Nudity, Penetration :  ['Nudity', 'Penetration']
sexual harassment, Sexual remarks :  ['sexual harassment', 'Sexual remarks']
Sexual humiliation, Sexual extortion, Prostitution/sex work :  ['Sexual humiliation', 'Sexual extortion', 'Prostitution', 'sex work']
Taser, Baton, physical touch, gun :  ['Taser', 'Baton', 'physical touch', 'gun']
Trespass/robbery, Racial slurs, xenophobic remarks :  ['Trespass', 'robbery', 'Racial slurs', 'xenophobic remarks']
Undocumented status, ICE :  ['Undocumented status', 'ICE']
Planting drug, planting guns :  ['Planting drug', 'planting guns']
Neglect of duty, failure to serve :  ['Neglect of duty', 'failure to serve']
Refusing to provide medical assistance :  ['Refusing to provide medical assistance']
Irrational, Aggressive, Unstable :  ['Irrational', 'Aggressive', 'Unstable']
searching minors/patting down minors/arresting minors :  ['searching minors', 'patting down minors', 'arresting minors']


In [51]:
for t in terminology_list:
    terms = split_by_punc(t)
    patterns = [nlp.make_doc(text) for text in terms]
    matcher.add(t, None, *patterns)

In [42]:
testtext = "Taser, Baton, physical touch, gun"

In [52]:
doc = nlp(text1)
matches = matcher(doc)#print the matched results and extract out the results
for match_id, start, end in matches:
    # Get the string representation 
    string_id = nlp.vocab.strings[match_id]  
    span = doc[start:end]  # The matched span
    print("Category: {}, Indexes: {} to {}, Word: {}".format(string_id, start, end, span.text))

Category: Irrational, Aggressive, Unstable, Indexes: 3436 to 3437, Word: irrational
Category: Trespass/robbery, Racial slurs, xenophobic remarks, Indexes: 6034 to 6035, Word: ROBBERY
Category: Trespass/robbery, Racial slurs, xenophobic remarks, Indexes: 6438 to 6439, Word: Trespass
